In [1]:
# run in docker:
# sudo docker run -p 8890:8888 -v /home/tehdog/data/dev/2019/pic2kcal-cv-praktikum:/tf/notebooks --runtime=nvidia -it --rm tensorflow/tensorflow:1.12.0-gpu-py3 jupyter notebook --allow-root --notebook-dir=/tf/notebooks
# crashes in TF1.4!
# does not work with Arch Linux TF installation!

In [2]:
%cd ..

/tf/notebooks


In [3]:
!pip install tensorflow-hub
!pip install sentencepiece
!pip install tf-sentencepiece
!pip install tqdm

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  en_de_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1")
  embedded_text = en_de_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0630 21:11:42.692018 139825671636736 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [5]:
from pathlib import Path
import json
import re
import random
import heapq
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
from math import ceil

data_dir = Path("data")

In [6]:
# jq '.[]' fddb_data_v3.json > fddb_data_v3.jsonl
# jq 'select(([.Bilder[]|select(.title != "Noch kein Foto vorhanden.")]|length) > 0)' fddb_data_v3.jsonl | jq -s > fddb_data_v3_withimg.json
with open(str(data_dir / "fddb_data_v3_withimg.json"), encoding='utf-8') as f:
    fddb = json.load(f)
    # todo: make unique here
    out_names = [e["name"] for e in fddb]

# jq '.ingredients[]|select(.ingredient)|.ingredient' processed_data.jsonl | jq -s unique > ingredients.json
with open(str(data_dir / "recipes/ingredients.json"), encoding='utf-8') as f:
    _in_names = json.load(f)


In [7]:
def normalize_ingredient(ing: str):
    ing = re.sub(r"\(([^)])\)", "\g<1>", ing)  # remove stuff in parens
    ing = re.sub(r"\([^)]+\)", "", ing)  # remove stuff in parens
    ing = re.sub(r"(\d+,)?\d+ k?g\b", "", ing)  # remove xyz gram
    ing = re.sub(r",.*", "", ing)
    ing = re.sub(r"\bzum .*", "", ing)
    ing = re.sub(r"\boder\b.*", "", ing)
    ing = ing.strip()
    return ing

In [8]:
in_names = list({normalize_ingredient(ing) for ing in _in_names})
len(in_names)

21444

In [9]:
len(out_names)

122977

In [ ]:
#def Matcher():
#    def __init__(self, data_left, data_right, preproc_left, preproc_right):
#        pre_left = list(map(preproc_left, data_left))
#        self.left_vecs = list(zip(data_left, pre_left, get_sentence_vectors(pre_left)))
#        
#        pre_right = list(map(preproc_right, data_right))
#        self.right_vecs = list(zip(data_right, pre_right, get_sentence_vectors(pre_right)))

In [ ]:
def make_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [ ]:
def get_sentence_vectors(texts):
    bs = 10000
    ccount = len(texts)//bs
    chunks = make_chunks(texts, bs)
    if ccount >= 3:
        chunks = tqdm(chunks, total=ccount)
    for chunk in chunks:
        yield from session.run(embedded_text, feed_dict={text_input: chunk})

In [ ]:
def get_match(search: np.array, out_vecs):
    it = ((v[0], np.dot(v[1], search)) for v in out_vecs)
    res_list = heapq.nlargest(10, it, key=itemgetter(1))
    return res_list

In [ ]:
out_vecs = list(zip(out_names, get_sentence_vectors(out_names)))

In [ ]:
out_vecs[0][1].shape

(512,)

In [ ]:
in_names[0], out_names[0]

('HENGLEIN frischer Flammkuchenteig', 'Get Fruity Bar, Marvellous Mango')

In [ ]:
in_vecs = list(zip(in_names, get_sentence_vectors(in_names)))

In [ ]:
for ingredient, vec in random.sample(in_vecs, 100):
    res_list = get_match(vec, out_vecs)
    print("{} -> {}".format(ingredient, res_list[0]))

Piment d’Espelette -> ('Petit Frais au Curcuma', 0.86310613)
Baiser - Boden -> ('Ovalis - Paradeiser', 0.7531158)
Salsa Dip -> ('Salsa Dip', 1.0000001)
Smarties o. ä. -> ('Smarties', 0.774335)
WanTan - Teigblätter -> ('Teigblätter', 0.7666634)
Mandelstifte -> ('Mandelbällchen', 0.8798618)
Crème fraîche Legere -> ('Crème fraîche', 0.969875)
Dörrfleisch nach Belieben -> ('Dörrpflaumen', 0.8018552)
Hanfsamen -> ('Hanfsamen', 1.0)
Gehacktes / Mett -> ('Dietzel Mettenden', 0.7645625)
Gemüsebrühe aus den Schalen des Gemüses -> ('Gemüsebrühe, aus frischen zutaten', 0.9037679)
Zucker 2 -> ('Zucker', 0.86964095)
Sahnemeerrettich -> ('Sahnemeerrettich', 1.0)
Putenbrust -> ('Putenbrust', 1.0)
Salz und Pfeffer ... -> ('Pfeffer & Salz Chips, Pfeffer und Salz', 0.73837936)
Fleischbrühe von den Rippchen -> ('Rinderknochenbrühe', 0.859671)
Mini-Mozzarella Kugeln -> ('Mozzarella Kugel', 0.9120718)
Fischfond - Paste -> ('Fischfond', 0.8705851)
Zucker - Austauschstoff -> ('Flüssigsüßstoff', 0.7813295)
Mo

In [ ]:
z = [inv for inv in in_vecs if inv[0] == 'Kartoffeln'][0]
get_match(z[1], out_vecs)

In [ ]:
from multiprocessing import Pool
def get_match_w(t):
    ing, vec = t
    return ing, get_match(vec, out_vecs)

all_ings = {}
with Pool(8) as pool:
    for ing, match in pool.imap(get_match_w, tqdm(in_vecs), chunksize=100):
        all_ings[ing] = match

In [ ]:
def jsonable(e):
    return [(a,float(b)) for a, b in e]
all_ings_orig = {ing: jsonable(all_ings[normalize_ingredient(ing)]) for ing in _in_names}

In [ ]:
with open("data/recipe-ingredient-to-fooddb.json", "w") as f:
    json.dump(all_ings_orig, f)

In [ ]:
#


In [ ]:
[ov for ov in out_vecs if ov[0] == 'Butter']

In [ ]:
re.match("^Nährwerte für 100 (\w+)$", "Nährwerte für 100 g").groups()

In [ ]:
from collections import defaultdict
fddb_entries = defaultdict(list)
rem = re.compile(r"^(\d+ )?(.*) \((\d+(?:,\d+)?) (g|ml)\)$")
for e in fddb:
    fddb_entries[e["name"]].append(e)
    sanity = list(e["Standard Nährwerte"].keys())
    snmatch = re.match("^Nährwerte für 100 (\w+)$", sanity[0])
    if snmatch is not None:
        unn = snmatch.groups()[0]
        e["parsed"] = {"unit": unn, "for_100": e["Standard Nährwerte"]["Nährwerte für 100 {}".format(unn)]}
        
    else:
        print("SANITY ERROR:", e["Id"], sanity, unit)
        continue
    amts = e["parsed"]["amounts"] = []
    for k in e["Spezifische Nährwerte"].keys():
        if k == "100 g (100 g)" or k == "100 g (100 ml)" or k == "100 ml (100 ml)":
            continue
        txtamount, txt, amount, unit = rem.match(k).groups()
        amount = float(amount.replace(",", "."))
        txtamount = float(txtamount.replace(",", ".")) if txtamount else 1.0
        amts.append((txt, k, amount/txtamount))
        
        if unn != unit:
            raise Exception("noooo")
        #print(txt, amount/txtamount, unit)

In [ ]:
# get recipe meta
with open("data/recipes/processed_data.json", encoding="utf-8") as f:
    recipes = json.load(f)

In [ ]:
# get amount map from parse-amounts
with open("data/recipes/parsed_amounts.json") as f:
    parsed_amounts = json.load(f)

In [ ]:
# only recipes with pictures
recipes = [recipe for recipe in recipes if len(recipe['picture_files']) > 0]

In [ ]:
from operator import itemgetter
# ugly af code
debug=False
for recipe in random.sample(recipes, 10):
    print("----------------------------")
    print("" + recipe["title"])
    for ingredient in recipe["ingredients"]:
        if "amount" in ingredient:
            print((ingredient["amount"].rjust(15) + " " + ingredient["ingredient"]).ljust(55), end='')
            print(" = ", end='')
            _z = parsed_amounts[ingredient["amount"]]
            if _z is None:
                print(ingredient["amount"])
            amount_count, amount_type = _z
            
            if amount_count is None:
                print(" [ignored]")
                continue
            if amount_type is None:
                amount_type = "Stück"
            #print("[" + str(amount_count) + "*" + (amount_type)+ "] of ", end="")
            _ing_matches = all_ings_orig[ingredient["ingredient"]]
            ing_matches = [e for e in _ing_matches if e[1] > 0.85]
            if len(ing_matches) == 0:
                # not so great match but eh
                ing_matches = [_ing_matches[0]]
            possible_entries = {entry["Id"]: (ing_match[1], entry) for ing_match in ing_matches for entry in fddb_entries[ing_match[0]]}
            possible_entries = sorted(list(possible_entries.values()), key=lambda e: -e[0])

            debug and print("matches", ing_matches)
            debug and print("finding unit ", amount_type)
            match = None
            for accuracy, e in possible_entries:
                if "parsed" not in e:
                    continue
                debug and print("considering", e["name"], e["Id"])
                have_unit = [amount_type]
                want_unit = e["parsed"]["unit"]
                
                if want_unit in have_unit:
                    debug and print("got direct match!", amount_count, have_unit, amount_count, want_unit)
                    match = (e, accuracy, amount_count, want_unit, amount_count, want_unit)
                    break
                for amount_name, amount_source, to_norm in e["parsed"]["amounts"]:
                    debug and print(e["name"], "amount", amount_source)
                    if amount_name in have_unit:
                        debug and print("got match unit!", amount_source)
                        match = (e, accuracy, amount_count, have_unit[0], amount_count*to_norm, want_unit)
                        break
                if match is not None:
                    break
            if not match:
                print("[nomatch]")
            else:
                fddb_entry, accuracy, count_weird, unit_weird, count_norm, unit_norm = match
                if unit_weird == unit_norm:
                    unittxt = "{}{}".format(count_norm, unit_norm)
                else:
                    unittxt = "{} {} = {}{}".format(count_weird, unit_weird, count_norm, unit_norm)
                values = e["parsed"]["for_100"]
                value_kcal_per_100 = float(values["Kalorien"]["Menge"].replace(",", "."))
                value_kcal = value_kcal_per_100 / 100 * count_norm
                print("({:.0f}%) [{}] of {} = {:.0f} kcal".format(accuracy * 100, unittxt, fddb_entry["name"], value_kcal))
        else:
            print("->" + ingredient["subtitle"])